In [3]:
import torch 
import torch.utils.data as data
import torchvision
from skimage import io
import matplotlib.pyplot as plt


In [19]:
ecg_image=torchvision.datasets.ImageFolder('images/')
data_loader=torch.utils.data.DataLoader(ecg_image,batch_size=4,shuffle=True)

In [29]:
print(len(ecg_image))
print(ecg_image.classes)

6
['1']


In [ ]:
class mydataset(data.Dataset):
    def __init__(self, images, labels):
        self.images=images
        self.labels=labels
    def __getitem__(self,index):
        